In [ ]:
import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

import tensorflow_datasets as tfds

import warnings
warnings.simplefilter("ignore")

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [ ]:
print(ds_info)

In [ ]:
assert isinstance(ds_train, tf.data.Dataset)

one = ds_train.take(1) # one example is `{'image': tf.Tensor, 'label': tf.Tensor}`
image, label = list(tfds.as_numpy(one))[0]
print(type(image), type(label), label)
print("shape:", image.shape, "label:", label)


def show_images(ds):
    figure = plt.figure(figsize=(10,10))
    for i, example in enumerate(tfds.as_numpy(ds)):
        image, label = example
        if i + 1 > 16:
            break
        plt.subplot(4, 4, i + 1, title=label)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(image)

ds = ds_train.take(16)
show_images(ds)


In [ ]:
model = Sequential([
  tf.keras.layers.Input(shape=32*32*3),
  Dense(10000, activation='relu'),
  Dense(2000, activation='relu'),
  Dense(10)
])
model.summary()

# seems like 10000, 2000, 10 is not enough, loss flattens very quickly - it seems that model is too small.

In [ ]:
def translate_and_flatten(ds, no):
    X = np.zeros((no, 32, 32, 3))
    Y = np.zeros(no)
    for i, example in enumerate(tfds.as_numpy(ds)):
        image, label = example
        X[i] = image
        Y[i] = label
    X = X.reshape(no, -1)
    return X, Y

In [ ]:
NO_OF_EXAMPLES = 18000
ds = ds_train.take(NO_OF_EXAMPLES)
X, Y = translate_and_flatten(ds, NO_OF_EXAMPLES)
print(Y[:10])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(X, Y, epochs=3)


In [ ]:
Xt, Yt = translate_and_flatten(ds_test.take(100), 100)
pred = model.predict(Xt)

In [ ]:
cnt = 0
for i, val in enumerate(pred):
    pred_label = np.argmax(val)
    if (pred_label != int(Yt[i])):
        cnt += 1
print("accuracy:", 1 - cnt/len(pred))